<a href="https://colab.research.google.com/github/ShimaSama/Netflix-QoS/blob/main/google_colab/MLPClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import pandas
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
import random
import joblib

# Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = '/content/drive/My Drive/MALWIAS/TMA/'

Mounted at /content/drive


In [ ]:
def get_dataset_from_csv(filename):
  file = open(BASE_DIR + filename, 'r')
  custom_fields = {item: lambda x: int(x, 16) for item in ["tcp.flags", "tcp.checksum", "ip.checksum", "ip.dsfield", "ip.flags", "ip.id"]}
  custom_fields.update({'label': lambda x: float(x == "netflix")})
  dataset = pandas.read_csv(file, converters=custom_fields)
  values = dataset.values
  X = values[:,:len(values[1]) - 1]
  Y = values[:,len(values[1]) - 1]
  return X, Y

X, Y = get_dataset_from_csv('data/computer.csv')
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=0.3, random_state=random.randint(0, 100000))
X_test, Y_test = get_dataset_from_csv('data/test.csv')

# Create Model

## Loading model

In [ ]:
filename = BASE_DIR + 'models/MLPClassifier.sav'
model = joblib.load(filename)

## Creating model

In [ ]:
model = MLPClassifier(learning_rate_init=0.00000175, verbose=True, learning_rate='adaptive')
model.fit(X_train, Y_train)

Iteration 1, loss = 20.87274902
Iteration 2, loss = 6.15126468
Iteration 3, loss = 6.86002714
Iteration 4, loss = 9.25365564
Iteration 5, loss = 9.32391912
Iteration 6, loss = 9.34246717
Iteration 7, loss = 9.40542535
Iteration 8, loss = 9.32526904
Iteration 9, loss = 9.34223981
Iteration 10, loss = 9.24710296
Iteration 11, loss = 9.26218063
Iteration 12, loss = 9.12762826
Iteration 13, loss = 8.91937718
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(learning_rate='adaptive', learning_rate_init=1.75e-06,
              verbose=True)

## Saving model

In [ ]:
filename = BASE_DIR + 'models/MLPClassifier2.sav'
joblib.dump(model, filename)

['/content/drive/My Drive/MALWIAS/TMA/models/MLPClassifier2.sav']

# Test & Validation

In [ ]:
score = model.score(X_validation, Y_validation)
expected = Y_test
predicted = model.predict(X_test)

print("Model Score: %s" % score)

Model Score: 0.6941669192747159


In [ ]:
th = 0.5
false_positive = false_negative = true_positive = true_negative = good = 0
for predicted_i, expected_i in zip(predicted, expected):
  predicted_val = int(predicted_i >= th)
  if predicted_val == int(expected_i):
    good += 1
    if predicted_val == 0:
      true_negative += 1
    else:
      true_positive += 1
  else:
    if predicted_val == 0:
      false_negative += 1
    else:
      false_positive += 1

print("true positive = " + str(true_positive)+ "---> rate = " + str(true_positive/(true_positive+false_negative)))
print("true negative = " + str(true_negative) + "---> rate = " + str(true_negative/(true_negative+false_positive)))
print("false negative = " + str(false_negative) + "---> rate = " + str(false_negative/(false_negative+true_positive)))
print("false positive = " + str(false_positive) + "---> rate = " + str(false_positive/(false_positive+true_negative)))
print("sensitivity = " + str(true_positive/(true_positive+false_negative)))
print("specifity = " + str(true_negative/(true_negative+false_positive)))
print("acurracy = " + str(good/len(predicted)))

true positive = 1550---> rate = 0.5759940542549238
true negative = 14304---> rate = 0.710934393638171
false negative = 1141---> rate = 0.42400594574507616
false positive = 5816---> rate = 0.28906560636182904
sensitivity = 0.5759940542549238
specifity = 0.710934393638171
acurracy = 0.6950155626671343
